In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install patool

     |████████████████████████████████| 77 kB 6.0 MB/s 


In [5]:
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/gaze360.rar", outdir="/content")

patool: Extracting /content/drive/MyDrive/gaze360.rar ...
patool: running /usr/bin/unrar x -- /content/drive/MyDrive/gaze360.rar
patool:     with cwd='/content'
patool: ... /content/drive/MyDrive/gaze360.rar extracted to `/content'.


'/content'

In [6]:
import numpy as np
from os.path import dirname, join as pjoin
import scipy.io as sio
import cv2
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.patches as patches
import math
from math import *
from torch.utils.data import Dataset
import torchvision
import torch.optim as optim

source_mat = ''
#=r"C:\Users\108652706\Documents\DOCUMENT_perso\ISI\MLA\trials\gaze360"
mat_fname = pjoin(source_mat, '/content/gaze360/metadata.mat')
mat_contents = sio.loadmat(mat_fname)
sorted(mat_contents.keys())

['__globals__',
 '__header__',
 '__version__',
 'frame',
 'gaze_dir',
 'person_body_bbox',
 'person_cam',
 'person_eye_left_bbox',
 'person_eye_right_bbox',
 'person_eyes2d',
 'person_eyes3d',
 'person_face_bbox',
 'person_head_bbox',
 'person_identity',
 'recording',
 'recordings',
 'split',
 'splits',
 'target_cam',
 'target_pos2d',
 'target_pos3d',
 'ts']

In [7]:
#name= mat_contents['']

recordings=mat_contents['recordings']
recording = mat_contents['recording']
person_identity= mat_contents['person_identity']
frame= mat_contents['frame']
person_head_bbox = mat_contents['person_head_bbox']
person_eye_left_bbox= mat_contents['person_eye_left_bbox']
cropType= 'head'
person_face_bbox = mat_contents['person_face_bbox']
target_pos3d=mat_contents['target_pos3d']
person_eye_right_bbox= mat_contents['person_eye_right_bbox']
gaze_dir =mat_contents['gaze_dir']

In [8]:
!pip install face_alignment # used as second checker of eye detection in a head image

  Created wheel for face-alignment: filename=face_alignment-1.3.5-py2.py3-none-any.whl size=28239 sha256=8b278c5f249d0937e41a6dbaf81255f4e37db150716d0959b02f17262d873003
  Stored in directory: /root/.cache/pip/wheels/c9/ba/4d/2d368f55e5f929f9472da59e356fbdf1483f885de80a5bc620
Successfully built face-alignment


In [9]:
import face_alignment
from skimage import io
from mpl_toolkits.mplot3d import Axes3D
import collections
# Optionally set detector and some additional detector parameters
face_detector = 'sfd'
face_detector_kwargs = {
    "filter_threshold" : 0.8
}

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, device='cpu', flip_input=True,
                                  face_detector=face_detector, face_detector_kwargs=face_detector_kwargs)

Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth


  0%|          | 0.00/85.7M [00:00<?, ?B/s]

Downloading: "https://www.adrianbulat.com/downloads/python-fan/3DFAN4-4a694010b9.zip" to /root/.cache/torch/hub/checkpoints/3DFAN4-4a694010b9.zip


  0%|          | 0.00/91.9M [00:00<?, ?B/s]

Downloading: "https://www.adrianbulat.com/downloads/python-fan/depth-6c4283c0e0.zip" to /root/.cache/torch/hub/checkpoints/depth-6c4283c0e0.zip


  0%|          | 0.00/224M [00:00<?, ?B/s]

In [10]:
from torchvision import models 
from torchvision import transforms
import torch.nn as nn
from PIL import Image
from matplotlib import cm
import torch
from torch.utils.data.sampler import SubsetRandomSampler

In [11]:
resnet = models.resnet18(pretrained=True)
resnet18 = nn.Sequential(*(list(resnet.children())[0:8])) #take 8 layers 
#resnet18.cpu()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

#create functions

In [12]:
#first check  of eye detection
def  Kellnhofer_al_eye(imHead,headBBInFull,eyeBBInFull): 
        cropSizePx = [imHead.shape[1], imHead.shape[0]]
        #eyeBBInFull = person_eye_left_bbox[i,:]
        eyeBBInCrop = [
            (eyeBBInFull[0] - headBBInFull[0]) / headBBInFull[2], # subtract offset of the crop
            (eyeBBInFull[1] - headBBInFull[1]) / headBBInFull[3], 
            eyeBBInFull[2] / headBBInFull[2], # scale to smaller space of the crop
            eyeBBInFull[3] / headBBInFull[3], 
            ]
        eyeBBInCropPx_l = np.concatenate([np.multiply(eyeBBInCrop[:2], cropSizePx), np.multiply(eyeBBInCrop[2:], cropSizePx)]).astype(int)
        imEye_l = imHead[
            eyeBBInCropPx_l[1]:(eyeBBInCropPx_l[1]+eyeBBInCropPx_l[3]), 
            eyeBBInCropPx_l[0]:(eyeBBInCropPx_l[0]+eyeBBInCropPx_l[2]),
            :]
        fl= imEye_l.flatten()
        imEye_l = cv2.resize(imEye_l,(60,36)) 
        return imEye_l

In [13]:
#1st check of eye detection : using the technique of kellnhofer et al article, detect eyes if possible, else, return zeros 
def Kellnhofer_al_eyes(imHead,head_box,eye_left_box,eye_right_box):
    if np.all(eye_left_box==np.array([-1,-1,-1,-1])):
        imEye_l=np.zeros((36,60,3), np.uint8)
    else:
        imEye_l = Kellnhofer_al_eye(imHead,head_box,eye_left_box)

    if np.all(eye_right_box==np.array([-1,-1,-1,-1])):
        imEye_r= np.zeros((36,60,3), np.uint8)
    else:
        imEye_r = Kellnhofer_al_eye(imHead,head_box,eye_right_box)
    #return fl,fr
    return imEye_l,imEye_r

In [62]:
#second check : using the 3D technique of bulat et al article, detect eyes, else return zeros
def bulat_al(imHead,fa):#second check
    try : 
      preds = fa.get_landmarks(imHead)[-1]#predict
      # 2D-Plot
      pred_type = collections.namedtuple('prediction_type', ['slice', 'color'])
      pred_types = {
                'eye1': pred_type(slice(36, 42), (0.596, 0.875, 0.541, 0.3)),
                'eye2': pred_type(slice(42, 48), (0.596, 0.875, 0.541, 0.3)),
                }
      centers=[]
      Avr_eye=24 #24 represents average eye length for adults, we use this to set the scale

      for pred_type in pred_types.values():
          x=preds[pred_type.slice, 0]
          y=preds[pred_type.slice, 1]
          centroid = (sum(x) / len(x), sum(y) / len(y)) #get centroid
          centers.append(centroid) #append
      dist = math.hypot(centers[0][0]-centers[1][0],centers[0][1]-centers[1][1]) #distance on image
      dist_reel= np.divide(dist*24,x.max()-x.min() )#real distance
      if (77 >dist_reel>51):  #normal distance between pupils is between 51 and 77
          centers = [(int(element[0]), int(element[1])) for element in centers]
          imEye_r=imHead[centers[0][0]-7:centers[0][0]+7,centers[0][1]-7:centers[0][1]+7,:]
          imEye_l=imHead[centers[1][0]-7:centers[1][0]+7,centers[1][1]-7:centers[1][1]+7,:]
          fl= imEye_l.flatten()
          fr= imEye_r.flatten()
          imEye_r = cv2.resize(imEye_r,(60,36)) 
          imEye_l = cv2.resize(imEye_l,(60,36)) 
      else :
          blank_image = np.zeros((36,60,3), np.uint8)
          imEye_l = blank_image
          imEye_r = blank_image
    except : 
      blank_image = np.zeros((36,60,3), np.uint8)
      imEye_l = blank_image
      imEye_r = blank_image
    return imEye_l,imEye_r


In [63]:
# extract eyes from head image function
def head_eye_extractors(imHead,fa,head_box,target,eye_left_box,eye_right_box):
      if np.all(eye_left_box==np.array([-1,-1,-1,-1])) and np.all(eye_right_box==np.array([-1,-1,-1,-1])):
          print("eye_not_detected with Kellnhofer_al ")
          imEye_l,imEye_r= bulat_al(imHead,fa)
      else: 
          imEye_l,imEye_r= Kellnhofer_al_eyes(imHead,head_box,eye_left_box,eye_right_box)
      x,y,z=target
      yaw = atan(np.divide(z,x))
      pitch=atan(np.divide(x,y))
      h=[float(yaw),float(pitch)]
      return h, imEye_l,imEye_r

In [64]:
from torchvision import models 
from torchvision import transforms
import torch.nn as nn
from PIL import Image
from matplotlib import cm
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import datasets
import math
device = torch.device("cpu")

In [65]:
from torchvision import models
from torchsummary import summary
summary(resnet18,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

#form datasets

In [66]:
def dataset ( source_p,nb_image):
    head_imgs=[]
    l_imgs=[]
    r_imgs=[]
    heads=[]
    h_list=[]
    gaze=[]
    for i in range(nb_image): 
          imHead = cv2.imread(os.path.join(
            source_p,
            recordings[0,recording[0,i]][0],
            cropType,
            '%06d' % person_identity[0,i],
            '%06d.jpg' % frame[0,i]
            ))
          heads.append(cv2.resize(imHead,(224,224)))
          h,imEye_l,imEye_r= head_eye_extractors(imHead,fa,person_head_bbox[i,:],target_pos3d[i],person_eye_left_bbox[i,:],person_eye_right_bbox[i,:])
          l_imgs.append(imEye_l)
          r_imgs.append(imEye_r)
          h_list.append(h)
          gaze.append(gaze_dir[i])
    return l_imgs,r_imgs,h_list,heads,gaze



In [ ]:
source_p=('/content/gaze360/imgs') #path to dataset
nb_image= len(frame[0])# number of images
nb_image = 2000 #trial with 20 images only
l_imgs,r_imgs,h,heads,gazes=dataset ( source_p,nb_image)

eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
e

/usr/local/lib/python3.7/dist-packages/face_alignment/api.py:144: UserWarning: No faces were detected.
  warnings.warn("No faces were detected.")


eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
eye_not_detected with Kellnhofer_al 
e

In [ ]:
'''
l_imgs is fl 
r_imgs is fr 
h is (yaw,pich) list 
heads is heads dataset
gazes is (gx,gy,gz) a.k.a target 
'''

## Eye prediction

In [ ]:
class eyeDataset(Dataset):
    """eye landmark dataset."""

    def __init__(self, imgs, transform=None):
       
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        #img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[idx, 0])
        image = self.imgs[idx] #io.imread(img_name)
        image = Image.fromarray(image)
        sample = {'image': image}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [ ]:
preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])

In [ ]:
l_eye_set=eyeDataset(imgs=l_imgs,transform=preprocess)
l_eye_loader = DataLoader(l_eye_set, batch_size=nb_image,
                        shuffle=False, num_workers=0)

l=next(iter(l_eye_loader))
outputs_l=l['image'].to(device)
resnet18.eval()

features_left=resnet18(outputs_l).flatten(start_dim=1) # yields a tensor of size([nb_image, 2048])


In [ ]:
features_left.shape

In [ ]:
r_eye_set=eyeDataset(imgs=r_imgs,transform=preprocess)
r_eye_loader = DataLoader(r_eye_set, batch_size=nb_image,
                        shuffle=False, num_workers=0)

r=next(iter(r_eye_loader))
outputs_r=r['image'].to(device)
resnet18.eval()

features_right=resnet18(outputs_r).flatten(start_dim=1)# yields a tensor of size([n_images, 2048])

In [ ]:
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
summary(resnet18.to(device),input_size=(3, 36, 60))# summary works only with gpu,so we change model to gpu then take it back to cpu
device= torch.device('cpu')


##Head pose

In [ ]:

class headposeDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, imgs, h, transform=None):
        self.h = h
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.imgs[idx] 
        image = Image.fromarray(image)
        ex = self.h[idx]
        '''
        landmarks = np.zeros(2)
        landmarks=np.zeros(2)
        landmarks[0]= ex[0]
        landmarks[1]= ex[1]
        '''
        landmarks = np.array(ex)
        if self.transform:
            image = self.transform(image)
        #sample = {'image': image, 'landmarks': landmarks}
        sample = {'image': image, 'landmarks': landmarks}

        return sample

In [ ]:
preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.double()),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])
face_pose_dataset = headposeDataset(imgs=heads,h=h,transform=preprocess)
dataloader = DataLoader(face_pose_dataset, batch_size=4,
                        shuffle=True, num_workers=0)


In [ ]:
validation_split = .2
shuffle_dataset = True
random_seed= 42
batch_size= 4

# Creating data indices for training and validation splits:
dataset_size = len(face_pose_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(face_pose_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(face_pose_dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [ ]:
#Function to create a Pretrained Resnet Model with a Fully Connected Layer
# (out_1: ouput of 1st layer), (out_2: ouput of 2nd layer)
def createModel_resnet34(out_1, out_2):
    model = torchvision.models.resnet34(pretrained = True)
    num_ftrs = model.fc.in_features

    for param in model.parameters():
        param.requires_grad = False
    
    #Creating 3 Linear connected layers which can be trained
    fc1 = nn.Linear(num_ftrs, out_1)
    fc2 = nn.Linear(out_1, out_2)
    fc3 = nn.Linear(out_2, 2)

    layers = [fc1, fc2, fc3]
    for linearLayer in layers:
        #Applying He initialization to all layers
        nn.init.kaiming_uniform_(linearLayer.weight, nonlinearity='leaky_relu')
  


    #Setting Resnet's fully connected layer to our collection of three Linear layers with nn.Sequential
    model.fc = nn.Sequential(fc1, nn.LeakyReLU(),fc2, nn.LeakyReLU(),fc3)
    model.double() #double to set variables to double
    #Sending the device to the GPU if avaliable
    model.to(device)

    return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#summary(m.to(device),input_size=(3, 224, 224))
device= torch.device('cpu')


In [ ]:
#check types : make sure both types are the same
print(next(iter(train_loader))['image'].type())
print(next(iter(train_loader))['landmarks'].type())


In [ ]:
for batch_idx, (data_, target_) in enumerate(train_loader):
        print(batch_idx,data_,target_)

In [ ]:
#Function to handle training
#(model: model to train), (criterion: loss function for the model), (optimizer, what optimizer to use)
def train(model, criterion, optimizer, file, epochs=9, trainLoss=[], validationLoss=[]):
    device = torch.device("cpu")
    print("Starting training")
    for epoch in range(epochs):
        print(f"Epoch Number: {epoch}")

        testAccuracy = 0
        totalLoss = 0

        for i in range(len(train_loader)):
            #Put model into training mode
            model.train()
            batch= next(iter(train_loader))
            x=batch['image']
            y=batch['landmarks']
            #Zero out gradients
            optimizer.zero_grad()
            #Make a prediction
            y_hat = model(x)
            
            #Calculate the loss
            loss = criterion(y_hat, y)
            l = torch.sqrt(loss)
            #Update the total loss
            totalLoss += l.item()
            #Initiate backpropagation
            l.backward()
            
            #Update parameters
            optimizer.step()

            #testAccuracy += (y_hat == y).sum().item()
        trainLoss.append(totalLoss)
        print(f"loss: {totalLoss}")
        
        valAccuracy = 0
        totalLoss = 0
        
        
        for j in range(len(validation_loader)):
            # Put the model into evaluation mode
            banch= next(iter(validation_loader))
            x=banch['image']
            y=banch['landmarks']
            model.eval()
            y_pred= model(x)
            print(y_pred)
            loss = criterion(y_pred, y)
            l = torch.sqrt(loss)
            totalLoss += l
        print(f"val loss: {totalLoss}")
        validationLoss.append(totalLoss)
        #Save our model for each epoch
    fig = plt.figure(figsize=(20,10))
    plt.title("Train-Validation rmse")
    plt.plot(trainLoss, label='train')
    plt.plot(validationLoss, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('accuracy', fontsize=12)
    plt.legend(loc='best')


    torch.save(model.state_dict(), file)
    return trainLoss, validationLoss, model

#Create a model using our function
model = createModel_resnet34(512, 128)
#Define our loss and optimizers
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr = 0.001)
#Specify output file name
file = "ResnetFinal.pth" #save model

#Try to use Colab's GPU here, otherwise it will take a long time to train.
output = train(model, criterion, optimizer, file)

In [ ]:
h

In [ ]:
%reset # to reset all variables